In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/users/akshay/PCPpred/RRCK/data/CycPeptMPDB_Peptide_Assay_RRCK (1).csv')
df.shape

(186, 247)

In [3]:
last_id = df['ID'].max()
print('last_id: ', last_id)

last_id:  5668


In [4]:
#Step 1: Removing unreliable datapoints (datapoints with Permeability value <= -10) and selecting the required Columns
df = df[df['Permeability'] > -10]
selected_columns = ['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']
new_df = df[selected_columns]
new_df.shape

(186, 5)

In [5]:
new_df.head(10)

,ID,SMILES,Permeability,Sequence,MolWt
0,22,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...,-5.96,"['Abu', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', '...",1202.635
1,23,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-5.74,"['L', 'dL', 'L', 'L', 'dP', 'Y']",712.933
2,24,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](C...,-6.30,"['dL', 'L', 'L', 'L', 'dP', 'Y']",712.933
3,25,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.31,"['L', 'Me_dL', 'meL', 'L', 'dP', 'meY']",755.014
4,26,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[C@@H...,-5.39,"['Me_dL', 'meL', 'L', 'meL', 'dP', 'meY']",769.041
5,27,CC(C)C[C@H]1C(=O)N(C)[C@H](CC(C)C)C(=O)N(C)[C@...,-5.46,"['meL', 'Me_dL', 'meL', 'meL', 'dP', 'meY']",783.068
6,28,CC(C)C[C@H]1C(=O)N(C)[C@@H](CC(C)C)C(=O)N2CCC[...,-5.21,"['Me_dL', 'meL', 'meL', 'meL', 'dP', 'meY']",783.068
7,29,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[C@@H...,-5.72,"['S', 'Me_dL', 'meL', 'L', 'dP', 'meY']",728.932
8,30,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.80,"['L', 'Me_dL', 'meS', 'L', 'dP', 'meY']",728.932
9,31,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-6.12,"['L', 'Me_dL', 'meL', 'S', 'dP', 'meY']",728.932


In [6]:
#Step 2: curating duplicate dapaoints based on SMILES representation
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]
non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
print("DataFrame with duplicates:")
print(duplicates)
print("\nDataFrame without duplicates:")
print(non_duplicates)
duplicates.to_csv('/home/users/akshay/PCPpred/RRCK/data/duplicates_RRCK.csv', index=False)

DataFrame with duplicates:
       ID                                             SMILES  Permeability  \
0      22  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...         -5.96   
1      23  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...         -5.74   
3      25  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...         -5.31   
8      30  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...         -5.80   
10     32  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...         -5.31   
11     33  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...         -5.81   
14     36  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...         -5.26   
21     43  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...         -6.74   
39    980  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...         -5.52   
40    981  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...         -5.96   
68   1862  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...         -5.25   
122  2326  CC(C)C[C@@H]1NC(=O)[C@H](C

In [7]:
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]

non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
#Permeability and MolWt of duplicate SMILES were averaged, and Most abundandant sequence was chosen
grouped_duplicates = duplicates.groupby('SMILES').agg(
    Permeability=('Permeability', 'mean'),   
    MolWt=('MolWt', 'mean'),   
    Sequence=('Sequence', lambda x: x.mode()[0]), 
).reset_index()

#Unique ID's were given to these curated datapoints, where ID's start from the last documented id in CycPeptMPDB_Peptide_Assay_PAMPA (5)
grouped_duplicates['ID'] = range(last_id + 1, last_id + 1 + len(grouped_duplicates))

grouped_duplicates = grouped_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]

non_duplicates_ordered = non_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]
merged_df = pd.concat([non_duplicates_ordered, grouped_duplicates], ignore_index=True)

print("DataFrame with processed duplicates:")
print(grouped_duplicates)

print("\nDataFrame without duplicates:")
print(non_duplicates)

print("Merged DataFrame:")
print(merged_df)

grouped_duplicates.to_csv('/home/users/akshay/PCPpred/RRCK/data/grouped_duplicates_RRCK.csv', index=False)
non_duplicates.to_csv('/home/users/akshay/PCPpred/RRCK/data/non_duplicates_RRCK.csv', index=False)
merged_df.to_csv('/home/users/akshay/PCPpred/RRCK/data/CycPeptMPDB_data_Main_RRCK.csv', index=False)

DataFrame with processed duplicates:
     ID                                             SMILES  Permeability  \
0  5669  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...        -5.760   
1  5670  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...        -5.225   
2  5671  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...        -5.255   
3  5672  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...        -6.185   
4  5673  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...        -5.805   
5  5674  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...        -5.805   

                                            Sequence     MolWt  
0  ['Abu', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', '...  1202.635  
1            ['L', 'Me_dL', 'meA', 'L', 'dP', 'meY']   712.933  
2            ['L', 'Me_dL', 'meL', 'L', 'dP', 'meY']   755.014  
3         ['L', 'Me_dL', 'Me_Cha', 'L', 'dP', 'meY']   795.079  
4            ['L', 'Me_dL', 'meS', 'L', 'dP', 'meY']   728.932  
5                   ['L', 'dL', 'L', 'L'

In [8]:
#Step 3: Splitting the Main dataset into Train and Test in ration 0.8:0.2
df = pd.read_csv('/home/users/akshay/PCPpred/RRCK/data/CycPeptMPDB_data_Main_RRCK.csv')
df

,ID,SMILES,Permeability,Sequence,MolWt
0,24,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](C...,-6.300,"['dL', 'L', 'L', 'L', 'dP', 'Y']",712.933
1,26,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[C@@H...,-5.390,"['Me_dL', 'meL', 'L', 'meL', 'dP', 'meY']",769.041
2,27,CC(C)C[C@H]1C(=O)N(C)[C@H](CC(C)C)C(=O)N(C)[C@...,-5.460,"['meL', 'Me_dL', 'meL', 'meL', 'dP', 'meY']",783.068
3,28,CC(C)C[C@H]1C(=O)N(C)[C@@H](CC(C)C)C(=O)N2CCC[...,-5.210,"['Me_dL', 'meL', 'meL', 'meL', 'dP', 'meY']",783.068
4,29,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[C@@H...,-5.720,"['S', 'Me_dL', 'meL', 'L', 'dP', 'meY']",728.932
...,...,...,...,...,...
171,5670,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.225,"['L', 'Me_dL', 'meA', 'L', 'dP', 'meY']",712.933
172,5671,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.255,"['L', 'Me_dL', 'meL', 'L', 'dP', 'meY']",755.014
173,5672,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-6.185,"['L', 'Me_dL', 'Me_Cha', 'L', 'dP', 'meY']",795.079
174,5673,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.805,"['L', 'Me_dL', 'meS', 'L', 'dP', 'meY']",728.932


In [9]:
#Main dataset is sorted based on Molecular weight in descending order
df_main = df.sort_values(by='MolWt', ascending= False)
df_main

,ID,SMILES,Permeability,Sequence,MolWt
133,2352,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H]([C...,-6.34,"['T', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', 'me...",1218.634
138,2358,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](C(...,-6.13,"['V', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', 'me...",1216.662
139,2359,C/C=C/C[C@@H](C)C(=O)[C@H]1C(=O)N[C@@H](C(C)C)...,-6.66,"['V', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', 'me...",1214.646
137,2357,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...,-5.95,"['Abu', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', '...",1202.635
140,2360,C/C=C/C[C@@H](C)[C@H]1OC(=O)[C@H](C(C)C)N(C)C(...,-6.78,"['Me_Bmt(E)', 'Abu', 'Sar', 'meL', 'V', 'meL',...",1202.635
...,...,...,...,...,...
91,2306,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H]2CCCN...,-4.75,"['L', 'Me_dL', 'meA', 'L', 'dP', 'meA']",620.836
116,2334,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-5.58,"['L', 'dA', 'A', 'L', 'dP', 'F']",612.772
90,2305,CCC[C@@H]1C(=O)N(C)[C@@H](C)C(=O)N[C@@H](CC(C)...,-4.85,"['L', 'Me_dNva', 'meA', 'L', 'dP', 'meA']",606.809
89,2304,CC[C@@H]1C(=O)N(C)[C@@H](C)C(=O)N[C@@H](CC(C)C...,-5.51,"['L', 'Me_dAbu', 'meA', 'L', 'dP', 'meA']",592.782


In [10]:
df_main['MolWt']

133    1218.634
138    1216.662
139    1214.646
137    1202.635
140    1202.635
         ...   
91      620.836
116     612.772
90      606.809
89      592.782
88      578.755
Name: MolWt, Length: 176, dtype: float64

In [11]:
#Every fifth datapoint is allocated to test set and remaining datapoints to Train set
test_mask = np.arange(len(df_main)) % 5 == 0
test_df = df_main[test_mask]
test_df.shape

(36, 5)

In [12]:
train_df = df_main[~test_mask]
train_df.shape

(140, 5)

In [13]:
print(train_df["Permeability"].max())
print(train_df["Permeability"].min())
print(train_df.shape)

-4.0
-7.48
(140, 5)


In [14]:
print(test_df["Permeability"].max())
print(test_df["Permeability"].min())
print(test_df.shape)

-4.62
-8.0
(36, 5)


In [15]:
test_df.to_csv("/home/users/akshay/PCPpred/RRCK/data/Test_RRCK.csv",index=False)
train_df.to_csv("/home/users/akshay/PCPpred/RRCK/data/Train_RRCK.csv",index=False)

In [16]:
train_df

,ID,SMILES,Permeability,Sequence,MolWt
138,2358,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](C(...,-6.13,"['V', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', 'me...",1216.662
139,2359,C/C=C/C[C@@H](C)C(=O)[C@H]1C(=O)N[C@@H](C(C)C)...,-6.66,"['V', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', 'me...",1214.646
137,2357,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...,-5.95,"['Abu', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', '...",1202.635
140,2360,C/C=C/C[C@@H](C)[C@H]1OC(=O)[C@H](C(C)C)N(C)C(...,-6.78,"['Me_Bmt(E)', 'Abu', 'Sar', 'meL', 'V', 'meL',...",1202.635
134,2353,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](C)...,-5.87,"['A', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', 'me...",1188.608
...,...,...,...,...,...
118,2336,CC[C@@H]1NC(=O)[C@@H](CC)NC(=O)[C@H](CC(C)C)NC...,-5.39,"['L', 'dAbu', 'Abu', 'L', 'dP', 'F']",640.826
91,2306,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H]2CCCN...,-4.75,"['L', 'Me_dL', 'meA', 'L', 'dP', 'meA']",620.836
116,2334,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-5.58,"['L', 'dA', 'A', 'L', 'dP', 'F']",612.772
90,2305,CCC[C@@H]1C(=O)N(C)[C@@H](C)C(=O)N[C@@H](CC(C)...,-4.85,"['L', 'Me_dNva', 'meA', 'L', 'dP', 'meA']",606.809


In [17]:
test_df

,ID,SMILES,Permeability,Sequence,MolWt
133,2352,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H]([C...,-6.340,"['T', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', 'me...",1218.634
170,5669,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...,-5.760,"['Abu', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', '...",1202.635
76,1881,CCCC[C@H]1C(=O)N(C)[C@H](CCCC)C(=O)N(C)[C@H](C...,-8.000,"['Me_Nva', 'Me_dAbu', 'Me_Nle', 'Me_dNle', 'Me...",1122.548
167,5666,CCCC[C@@H]1NC(=O)[C@H](CCCC)NC(=O)[C@H](CCCC)N...,-6.460,"['Me_dF', 'Nle', 'Me_Nle', 'Nle', 'Me_Nle', 'N...",1042.443
72,1877,CCCC[C@H]1C(=O)N(C)[C@H](CCCC)C(=O)N(C)[C@H](C...,-8.000,"['Me_dAbu', 'Me_Nle', 'Me_dNle', 'Me_dNle', 'M...",1009.388
68,1873,CC(C)C[C@H]1C(=O)N(C)[C@H](CC(C)C)C(=O)N2CCC[C...,-4.620,"['Me_dL', 'meA', 'Me_dL', 'Me_dA', 'meL', 'Me_...",953.280
73,1878,CCC[C@H]1C(=O)N(C)[C@H](CC)C(=O)N(C)[C@@H](C)C...,-7.300,"['Me_Nva', 'Me_dAbu', 'meA', 'Me_dAbu', 'Me_dA...",940.197
45,1849,O=C1CN(CCCc2ccccc2)C(=O)[C@H]2CCCN2C(=O)[C@H](...,-5.920,"['Nva(Ph)', '3Pal', 'dP', 'PhPr_Gly', 'Nva(Ph)...",918.152
52,1856,CC(C)(C)C[C@@H]1NC(=O)[C@@H](Cc2ccccc2)NC(=O)[...,-5.120,"['Ala(tBu)', 'Tza', 'dP', 'Mono48', 'Phe(4-F)'...",902.053
147,2367,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)...,-6.930,"['I', 'L', 'Me_dF', 'P', 'Sar', 'F', 'meY']",880.100


In [18]:
import pandas as pd

df = pd.read_csv('/home/users/akshay/PCPpred/RRCK/data/Train_RRCK.csv')

with open('/home/users/akshay/PCPpred/RRCK/data/Train_RRCK.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("train.smi file has been created successfully.")

train.smi file has been created successfully.


In [19]:
df = pd.read_csv('/home/users/akshay/PCPpred/RRCK/data/Test_RRCK.csv')

with open('/home/users/akshay/PCPpred/RRCK/data/Test_RRCK.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("test.smi file has been created successfully.")

test.smi file has been created successfully.
